In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import string
import math
import pickle
pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)
np.random.seed(123)
#https://www.analyticsvidhya.com/blog/2019/01/guide-pytorch-neural-networks-case-studies/
from sklearn.metrics import accuracy_score
from sklearn import linear_model, metrics, model_selection, ensemble
from sklearn.neural_network import MLPClassifier


In [4]:
predict_play = pickle.load(open('models/predict_play.sav', 'rb'))
rush_reg = pickle.load(open('models/rush_reg.sav', 'rb'))
pass_reg = pickle.load(open('models/pass_reg.sav', 'rb'))
fourth_down_classifier = pickle.load(open('models/4th_down_classifier.sav', 'rb'))
field_goal_classifier = pickle.load(open('models/field_goal_made.sav', 'rb'))

In [3]:
def get_4th_down(row):
    if (row['down'] == 4):
        return True
    else:
        return False